We write a program to calculate discrepancy.

Given a finite set $F \subset \mathbb{T}^2$ and a set $A \subset \mathbb{T}^2$ with a well-defined area (aka Lebesque measure) $\lambda(A)$, the discrepancy of $F$ with respect to $A$ is defined as $$D(F,A) = |\lambda(A) - \frac{|F \cap A|}{|F|}|$$
The discrepancy of $F$ is defined as $$D(F) = sup_{\text{rectangles} R} D(F,R)$$

In [1]:
###################################################
import math
import time
###################################################

In [2]:
###################################################
# Helper functions for calculating discrepancy
###################################################

# Helper function to determine if a given point is inside a rectangle
# Input: tuple representing a point; two tuples representing diagonal vertices of rectangle
# Output: 0 if point outside rectangle, 1 if point inside rectangle, 2 if point on border of rectangle

def pt_in_R(point, p1, p2):
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    
    """
    print("p1 is " + str(x1) + ", " + str(y1))
    print("p2 is " + str(x2) + ", " + str(y2))
    print("point[0] is " + str(point[0]))
    if point[0] == x1:
        print("point[0] == x1")
    """
    
    if x1 <= x2 and y1 <= y2: #rectangle does no overflow mod 1
        if x1 < point[0] and point[0] < x2: #point inside rectangle
            if y1 < point[1] and point[1] < y2:
                return 1
        elif point[0] == x1 or point[0] == x2: #point on left/right boundary
            if y1 <= point[1] and point[1] <= y2:
                return 2
        elif point[1] == y1 or point[1] == y2: #point on top/bottom boundary
            if x1 <= point[0] and point[0] <= x2:    
                return 2
            
    elif x1 > x2 and y1 <= y2: #rectangle overflows horizontally
        if x1 < point[0] or point[0] < x2: #point inside rectangle
            if y1 < point[1] and point[1] < y2:
                return 1
        elif point[0] == x1 or point[0] == x2: #point on left/right boundary
            if y1 <= point[1] and point[1] <= y2:
                return 2
        elif point[1] == y1 or point[1] == y2: #point on top/bottom boundary
            if x1 <= point[0] or point[0] <= x2:    
                return 2
        
    elif x1 <= x2 and y1 > y2: #rectangle overflows vertically
        if x1 < point[0] and point[0] < x2: #point inside rectangle
            if y1 < point[1] or point[1] < y2:
                return 1
        elif point[0] == x1 or point[0] == x2: #point on left/right boundary
            if y1 <= point[1] or point[1] <= y2:
                return 2
        elif point[1] == y1 or point[1] == y2: #point on top/bottom boundary
            if x1 <= point[0] and point[0] <= x2:    
                return 2
            
    else: #rectangle overflows both horizontally and vertically (occupies corners of the torus)
        if x1 < point[0] or point[0] < x2: #point inside rectangle
            if y1 < point[1] or point[1] < y2:
                return 1
        elif point[0] == x1 or point[0] == x2: #point on left/right boundary
            if y1 <= point[1] or point[1] <= y2:
                return 2
        elif point[1] == y1 or point[1] == y2: #point on top/bottom boundary
            if x1 <= point[0] or point[0] <= x2:    
                return 2
            
    return 0



# Helper function to calculate |R \cup F|
# Input: list of all tuples; two tuples (x,y) representing diagonal vertices of rectangle
# Output: number of points strictly inside rectangle, number of points on boundary of rectangle

def num_points_in_R(F, p1, p2): 
    count_in = 0
    count_on = 0
    for point in F:
        point_in_R = pt_in_R(point, p1, p2)
        if point_in_R == 1:
            count_in += 1
        elif point_in_R == 2:
            count_on += 1
    return (count_in, count_on)
                

In [3]:
####################################
# Discrepancy function
####################################

# We calculate the discrepancy of a finite input set F in the 2D torus.
# Input: a list of tuples (x,y) representing coordinates in the 2D torus. Assume that every x-coordinate is distinct
#  from all other x-coordinates, and similar for y-coordinates.
#  Optional: boolean show_rect for the option to show coordinates of the rectangle R with maximum value of D(F,R). 
# Output: discrepancy of the input.
# Approach: Iterate through all possible rectangles formed from x and y coordinates of points in F. For each
#  rectangle R, find D(F,R). Keep track of the maximum value of D(F,R).

def discrepancy(F, show_rect = False):
    x_coords = list()
    y_coords = list()
    
    for point in F:
        x_coords.append(point[0])
        y_coords.append(point[1])
        
    size_F = len(F)
    sup_R = -1
    sup_x1, sup_y1, sup_x2, sup_y2 = -1, -1, -1, -1
        
    for x1 in x_coords:
        for x2 in x_coords:
            for y1 in y_coords:
                for y2 in y_coords:
                    if x1 <= x2 and y1 <= y2: #rectangle does no overflow mod 1
                        area = (x2-x1)*(y2-y1) 
                    elif x1 > x2 and y1 <= y2: #rectangle overflows horizontally
                        area = (1-abs(x1-x2))*(y2-y1) 
                    elif x1 <= x2 and y1 > y2: #rectangle overflows vertically
                        area = (x2-x1)*(1-abs(y1-y2)) 
                    else: #rectangle overflows horizontally and vertically
                        area = (1-abs(x1-x2))*(1-abs(y1-y2))
                        
                    pt_in, pt_on = num_points_in_R(F, (x1,y1), (x2,y2))
                    
                    """
                    print("(x1,y1): " + str(x1) + ", " + str(y1))
                    print("(x2,y2): " + str(x2) + ", " + str(y2))
                    print("pt_in = " + str(pt_in))
                    print("pt_on = " + str(pt_on))
                    """
                    
                    for i in range(pt_on+1):
                        DFR = abs(area - (pt_in+i) / size_F)
                        #print("DFR = " + str(DFR))
                        if DFR > sup_R:
                            sup_R = DFR
                            sup_x1, sup_y1, sup_x2, sup_y2 = x1, y1, x2, y2
    
    if(show_rect):
        print("Supremum achieved with rectangle: " 
          + "(" + str(sup_x1) + "," + str(sup_y1) + "), " 
          + "(" + str(sup_x2) + "," + str(sup_y2) + ")")
    return sup_R


In [4]:
##############################################################
# Testing discrepancy function on differently sized sets
##############################################################

# Generate finite set F = {i*u1 + j*u2: 0 <= i,j < N}.
# Input: u1, u2 tuples representing vectors; N upper bound on i,j
# Output: list of points in torus satisfying {i*u1 + j*u2: 0 <= i,j < N}

def make_f(u1, u2, N):
    F = list()
    x1 = u1[0]
    y1 = u1[1]
    x2 = u2[0]
    y2 = u2[1]
    
    for i in range(N):
        for j in range(N):
            x = (i*x1+j*x2)%1
            y = (i*y1+j*y2)%1
            pt = (x, y) #maybe should be using numpy mat mul for this? eh it's fine
            #print(str(pt[0]) + ", " + str(pt[1]))
            F.append(pt)
    return F


# Build finite set F_N = {i*u1 + j*u2: 0 <= i,j < N} off of F_M, M < N. This is mostly for slightly better efficiency.
# Input: u1, u2 tuples representing vectors; N upper bound on i,j, F_M, M
# Output: list of points in torus satisfying {i*u1 + j*u2: 0 <= i,j < N}
"""
def make_f_from(u1, u2, N, F_M, M):
    F = F_M
    x1 = u1[0]
    y1 = u1[1]
    x2 = u2[0]
    y2 = u2[1]
    
    for i in range(M,N):
        for j in range(N):
            x = (i*x1+j*x2)%1
            y = (i*y1+j*y2)%1
            pt = (x, y)
            F.append(pt)
            
    for i in range(N):
        for j in range(M,N):
            x = (i*x1+j*x2)%1
            y = (i*y1+j*y2)%1
            pt = (x, y)
            F.append(pt)
    return F
"""

# Calculate the discrepancies of F_n = {i*u1 + j*u2: 0 <= i,j < n} for all N1 <= n <= N2.
# Input: u1, u2 tuples representing vectors; N1, N2
# Output: list of discrepancies of F_n for all N1 <= n <= N2

def disc_loop(u1, u2, N1, N2, show = False):
    disc = list()
    
    for i in range(N1,N2+1):
        tic = time.perf_counter()
        F = make_f(u1, u2, i)
        if show: 
            d = discrepancy(F, show_rect = True)
        else: 
            d = discrepancy(F)
        if show: print(str(i) + ": " + str(d))
        disc.append(d)
        toc = time.perf_counter()
        if show: print("time elapsed: " + str(toc-tic) + " sec")
    
    return disc
        
    

In [5]:
####################################
# main
####################################

# Testing discrepancy function.

def main():
    
    u1 = (1/math.sqrt(2),1/math.sqrt(3))
    u2 = (1/math.sqrt(4),1/math.sqrt(5))
    
    disc_loop(u1, u2, 1, 4, show = True)
    
    #F = make_f(u1,u2,4) 
    #print("The discrepancy of F is " + str(round(discrepancy(F, show_rect=True), 15)))
    

    
    
    

if __name__ == "__main__":
    main()

Supremum achieved with rectangle: (0.0,0.0), (0.0,0.0)
1: 1.0
time elapsed: 0.0006247500000426953 sec
Supremum achieved with rectangle: (0.7071067811865475,0.024563864689583825), (0.5,0.0)
2: 0.7734166970732304
time elapsed: 0.0028095420000227023 sec
Supremum achieved with rectangle: (0.0,0.15470053837925168), (0.7071067811865475,0.024563864689583825)
3: 0.5039751456284112
time elapsed: 0.06959679200002711 sec
Supremum achieved with rectangle: (0.7071067811865475,0.024563864689583825), (0.4142135623730949,0.8944271909999159)
4: 0.36508625673952233
time elapsed: 0.5358242089999976 sec


In [19]:
# Main issues: Resolved with round(_,15)
# F = [(0,0), (0.5,0.6), (0.25,0.75), (0.7,0.8)] should return 0.6. Instead, outputs 0.6000000000000001.
# F = [(0,0), (0.5,0.6), (0.25,0.75), (0.7,0.79)] should return 0.6075. Instead, outputs 0.6074999999999999.

# Appears to work correctly for:
# F = [(0,0)]
# F = [(0,0), (0.5,0.5)]
# F = [(0,0), (0.25,0.75)]